<a href="https://colab.research.google.com/github/ArunAmballa/Animal-Image-Classification-Using-CNN/blob/master/ResNet152V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U keras-tuner

     |████████████████████████████████| 98 kB 3.2 MB/s 


In [ ]:
from tensorflow import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Flatten
from keras.applications.resnet_v2 import ResNet152V2
from keras_tuner import RandomSearch
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from PIL import Image
from keras.callbacks import ReduceLROnPlateau

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
x_train=train_datagen.flow_from_directory(directory=r'/content/drive/MyDrive/Colab Data/Plant Disease/train', 
                                          target_size=(224,224), 
                                          class_mode='categorical',
                                          color_mode='rgb',
                                          batch_size=32)

Found 5702 images belonging to 3 classes.


In [ ]:
x_train.class_indices

{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}

In [ ]:
x_test=test_datagen.flow_from_directory(directory=r'/content/drive/MyDrive/Colab Data/Plant Disease/test', 
                                        target_size=(224,224),
                                        class_mode='categorical',
                                        color_mode='rgb',
                                        batch_size=32)

Found 1426 images belonging to 3 classes.


In [ ]:
IMAGE_SIZE=[224,224]
pretrained_model =ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

234561536/234545216 [==============================] - 2s 0us/step


In [ ]:
for layer in pretrained_model.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(pretrained_model.output)

In [ ]:
prediction = Dense(3, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
  def build_model(hp):
     model.compile(loss='categorical_crossentropy',
     optimizer=keras.optimizers.Adam(hp.Choice('learning rate',values=[1e-2,1e-3,1e-4,1e-5])),
                                     metrics=['accuracy'])
     return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5)

In [ ]:
tuner.search(x_train,epochs=5,batch_size=32,validation_data=x_test)

Trial 4 Complete [00h 06m 15s]
val_accuracy: 0.9943898916244507

Best val_accuracy So Far: 0.9943898916244507
Total elapsed time: 00h 45m 08s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning rate: 0.001
Score: 0.9943898916244507
Trial summary
Hyperparameters:
learning rate: 0.0001
Score: 0.9936886429786682
Trial summary
Hyperparameters:
learning rate: 0.01
Score: 0.9810659289360046
Trial summary
Hyperparameters:
learning rate: 1e-05
Score: 0.9719495177268982
